In [1]:
from IPython import display
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices

In [2]:
df = pd.read_csv("Datasets/seds.csv")
orig_num_rows = df.shape[0]
df.dropna(inplace=True)

print("Dropped {} rows".format(orig_num_rows - df.shape[0]))
print(df.shape)
display.display(df.describe())
display.display(df.head(10))

Dropped 28158 rows
(1683578, 8)


,year,value
count,1.683578e+06,1.683578e+06
mean,1.990484e+03,1.077022e+05
std,1.567905e+01,1.383675e+06
min,1.960000e+03,-7.090800e+05
25%,1.977000e+03,1.033000e+00
50%,1.991000e+03,8.400000e+01
75%,2.004000e+03,6.942000e+03
max,2.017000e+03,1.009913e+08


,year,msn,state_name,state_code,description,energy_bin,value,unit
0,1960,ABICB,Alaska,AK,Aviation gasoline blending components consumed...,Petroleum,0.0,Billion Btu
1,1961,ABICB,Alaska,AK,Aviation gasoline blending components consumed...,Petroleum,0.0,Billion Btu
2,1962,ABICB,Alaska,AK,Aviation gasoline blending components consumed...,Petroleum,0.0,Billion Btu
3,1963,ABICB,Alaska,AK,Aviation gasoline blending components consumed...,Petroleum,0.0,Billion Btu
4,1964,ABICB,Alaska,AK,Aviation gasoline blending components consumed...,Petroleum,0.0,Billion Btu
5,1965,ABICB,Alaska,AK,Aviation gasoline blending components consumed...,Petroleum,0.0,Billion Btu
6,1966,ABICB,Alaska,AK,Aviation gasoline blending components consumed...,Petroleum,0.0,Billion Btu
7,1967,ABICB,Alaska,AK,Aviation gasoline blending components consumed...,Petroleum,0.0,Billion Btu
8,1968,ABICB,Alaska,AK,Aviation gasoline blending components consumed...,Petroleum,0.0,Billion Btu
9,1969,ABICB,Alaska,AK,Aviation gasoline blending components consumed...,Petroleum,0.0,Billion Btu


In [3]:
asphalt_output = df.loc[df["msn"] == "ARTCB"]
display.display(asphalt_output.groupby("state_code")["value"].mean())

state_code
AK    3.383759e+03
AL    2.494067e+04
AR    1.032422e+04
AZ    1.762524e+04
CA    8.601350e+04
CO    1.778257e+04
CT    8.661086e+03
DC    7.453793e+02
DE    2.989948e+03
FL    3.432352e+04
GA    3.125991e+04
HI    2.172052e+03
IA    1.441405e+04
ID    7.916586e+03
IL    5.542566e+04
IN    3.827703e+04
KS    1.707222e+04
KY    1.821133e+04
LA    1.438660e+04
MA    1.219969e+04
MD    2.297645e+04
ME    4.640345e+03
MI    2.626553e+04
MN    3.342538e+04
MO    2.868505e+04
MS    1.454533e+04
MT    8.738328e+03
NC    2.661290e+04
ND    7.312448e+03
NE    6.490466e+03
NH    3.809241e+03
NJ    3.563652e+04
NM    9.914310e+03
NV    6.634966e+03
NY    4.026624e+04
OH    5.537586e+04
OK    2.201569e+04
OR    1.679934e+04
PA    4.286216e+04
RI    6.222345e+03
SC    1.437669e+04
SD    6.056810e+03
TN    2.728729e+04
TX    6.612184e+04
US    1.039574e+06
UT    9.436121e+03
VA    2.203609e+04
VT    1.933000e+03
WA    1.722953e+04
WI    2.575374e+04
WV    5.747638e+03
WY    6.241414e+03
N

In [7]:
# Preprocessing
df["year_categ"] = df["year"].astype("category")

TREAT_STATES_YEARS = {"AZ": 2006, "CA": 2002, "CO": 2004, "CT": 1998, "DE": 2005, "HI": 2001, "IL": 2007, "IN": 2011, "IA": 1983, "KS": 2009, "ME": 1999, "MD": 2004, "MA": 1997, "MI": 2008, "MN": 2007, "MO": 2007, "MT": 2005, "NV": 1997, "NH": 2007, "NJ": 1991, "NM": 2002, "NY": 2004, "NC": 2007, "ND": 2007, "OH": 2008, "OK": 2010, "OR": 2007, "PA": 2004, "RI": 2004, "SC": 2014, "SD": 2008, "TX": 1999, "UT": 2008, "VT": 2005, "VA": 2007, "WA": 2006, "WV": 2009, "WI": 1998, "DC": 2005}
TREAT_STATES = set(TREAT_STATES_YEARS.keys())
CTRL_STATES = {"GA", "KY", "ID", "AL", "MS", "FL", "LA", "WY", "TN", "NE", "AK", "AR"}
ALL_STATES = TREAT_STATES | CTRL_STATES

target_df = df[df["msn"] == "ARTCB"][["state_code", "year_categ", "value"]]
target_df.sort_values(["state_code", "year_categ"], inplace=True)
target_df.reset_index(drop=True, inplace=True)
display.display(target_df.head(10))

# time-fixed effects
year_df = pd.DataFrame()
year_df["year_categ"] = df["year_categ"].unique()
year_df["year_categ"] = year_df["year_categ"].astype("category")
year_df["key"] = 0

# unit-fixed effects
state_df = pd.DataFrame()
state_df["state_code"] = list(ALL_STATES)
state_df["state_code"] = state_df["state_code"].astype("category")
state_df["key"] = 0

# treatment indicator
regressors_df = pd.merge(year_df, state_df, on="key")
regressors_df.drop("key", axis=1, inplace=True)
regressors_df["treated"] = regressors_df.apply(
        lambda r: r["state_code"] in TREAT_STATES and r["year_categ"] > TREAT_STATES_YEARS[r["state_code"]],
        axis=1)
regressors_df.sort_values(["state_code", "year_categ"], inplace=True)
regressors_df.reset_index(drop=True, inplace=True)
display.display(regressors_df.head(10))

# preprocessed dataframe
prepped_df = pd.merge(regressors_df, target_df, on=["state_code", "year_categ"], sort=True)
display.display(prepped_df.head(10))

,state_code,year_categ,value
0,AK,1960,312.0
1,AK,1961,555.0
2,AK,1962,489.0
3,AK,1963,589.0
4,AK,1964,791.0
5,AK,1965,878.0
6,AK,1966,1646.0
7,AK,1967,832.0
8,AK,1968,755.0
9,AK,1969,969.0


,year_categ,state_code,treated
0,1960,AK,False
1,1961,AK,False
2,1962,AK,False
3,1963,AK,False
4,1964,AK,False
5,1965,AK,False
6,1966,AK,False
7,1967,AK,False
8,1968,AK,False
9,1969,AK,False


,year_categ,state_code,treated,value
0,1960,AK,False,312.0
1,1961,AK,False,555.0
2,1962,AK,False,489.0
3,1963,AK,False,589.0
4,1964,AK,False,791.0
5,1965,AK,False,878.0
6,1966,AK,False,1646.0
7,1967,AK,False,832.0
8,1968,AK,False,755.0
9,1969,AK,False,969.0


In [8]:
y, X = dmatrices("value ~ year_categ + state_code + treated", data=prepped_df, return_type="dataframe")
model = sm.RLM(y, X)
res = model.fit()
print(res.summary())

                    Robust linear Model Regression Results                    
Dep. Variable:                  value   No. Observations:                 2958
Model:                            RLM   Df Residuals:                     2849
Method:                          IRLS   Df Model:                          108
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Sat, 23 Feb 2019                                         
Time:                        11:27:35                                         
No. Iterations:                    10                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept          -2982.8475    948